# Debugging 0834+555 Calibration Issues

This notebook investigates the poor bandpass solutions reported for the 0834+555 observation.
We will:
1.  Confirm the transit time falls within the MS.
2.  Verify the phase center rephasing.
3.  Inspect the raw data for signal presence.
4.  Analyze the bandpass solutions.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import astropy.units as u
import casatools
from casacore.tables import table

tb = casatools.table()
ms = casatools.ms()

In [ ]:
# Path from the log
ms_path = '/stage/dsa110-contimg/ms/0834_555_2025-10-18_14-38-41.336.ms'

# If running in a different environment, check if we need to adjust
if not os.path.exists(ms_path):
    # Try local relative path if available
    potential_path = 'products/ms/0834_555_2025-10-18_14-38-41.336.ms'
    if os.path.exists(potential_path):
        ms_path = potential_path
    else:
        print(f"Warning: MS not found at {ms_path}")

print(f"Using MS: {ms_path}")

# 0834+555 Coordinates
src_ra = 128.728750 * u.deg
src_dec = 55.572500 * u.deg
source_coord = SkyCoord(ra=src_ra, dec=src_dec, frame='icrs')
print(f"Source: 0834+555 @ {source_coord}")

In [ ]:
# Open MS to get time range
t = table(ms_path, ack=False)
times = t.getcol('TIME')
t.close()

start_time = Time(np.min(times)/86400.0, format='mjd')
end_time = Time(np.max(times)/86400.0, format='mjd')
mid_time = Time((np.min(times) + np.max(times))/2.0/86400.0, format='mjd')

print(f"Observation Start: {start_time.iso}")
print(f"Observation End:   {end_time.iso}")
print(f"Duration:          {(end_time - start_time).to(u.min)}")

# OVRO Location
ovro = EarthLocation(lat=37.2317*u.deg, lon=-118.2951*u.deg, height=1222*u.m)

# Calculate AltAz at mid_time
altaz_frame = AltAz(obstime=mid_time, location=ovro)
src_altaz = source_coord.transform_to(altaz_frame)

print(f"Source Altitude at Mid-Time: {src_altaz.alt:.2f}")
print(f"Source Azimuth at Mid-Time:  {src_altaz.az:.2f}")

# Check if near transit (Az ~ 180 or 0 depending on Dec, but for DSA it's meridian)
# DSA-110 is a transit instrument, so HA should be close to 0.
# LST at OVRO
lst = mid_time.sidereal_time('apparent', longitude=ovro.lon)
ha = (lst - source_coord.ra).wrap_at(180*u.deg)

print(f"LST at Mid-Time: {lst:.2f}")
print(f"Hour Angle:      {ha:.2f}")

if abs(ha) < 15*u.deg: # Within ~1 hour
    print("CONFIRMED: Source is near transit.")
else:
    print("WARNING: Source is NOT near transit.")

In [ ]:
# Check FIELD table
t = table(f"{ms_path}/FIELD", ack=False)
phase_dirs = t.getcol('PHASE_DIR')
names = t.getcol('NAME')
t.close()

print(f"Number of fields: {len(names)}")
# Check if all phase dirs match the source
# PHASE_DIR shape is (nfields, 1, 2) usually (RA, Dec) in rad
ra_rad = src_ra.to(u.rad).value
dec_rad = src_dec.to(u.rad).value

for i in range(len(names)):
    p_ra = phase_dirs[i, 0, 0]
    p_dec = phase_dirs[i, 0, 1]
    
    sep = np.sqrt((p_ra - ra_rad)**2 * np.cos(dec_rad)**2 + (p_dec - dec_rad)**2)
    sep_arcmin = np.degrees(sep) * 60
    
    if sep_arcmin > 0.01:
        print(f"Field {i} ({names[i]}): Offset by {sep_arcmin:.4f} arcmin")
    else:
        pass # Good

print("Finished checking FIELD table.")

# Check UVW coordinates
# If rephased, UVW should be recalculated.
t = table(ms_path, ack=False)
uvw = t.getcol('UVW')
t.close()

u_coords = uvw[:, 0]
v_coords = uvw[:, 1]
w_coords = uvw[:, 2]

print(f"UVW Stats:")
print(f"  U range: {np.min(u_coords):.2f} to {np.max(u_coords):.2f}")
print(f"  V range: {np.min(v_coords):.2f} to {np.max(v_coords):.2f}")
print(f"  W range: {np.min(w_coords):.2f} to {np.max(w_coords):.2f}")

plt.figure(figsize=(6,6))
plt.plot(u_coords[::100], v_coords[::100], '.', markersize=1, alpha=0.5)
plt.xlabel('u (m)')
plt.ylabel('v (m)')
plt.title('UV Coverage (Subsampled)')
plt.axis('equal')
plt.show()

In [ ]:
# Plot Amp vs Time for a few baselines
ms.open(ms_path)
# Select a few baselines
ms.selectinit(datadescid=0) # SPW 0
# Get data
data = ms.getdata(['data', 'axis_info'], ifraxis=True)
ms.close()

vis = data['data'] # shape (npol, nchan, nbl)
# Average over channels and pols for quick look
amp = np.abs(vis)
mean_amp = np.mean(amp, axis=(0,1))

plt.figure(figsize=(10, 4))
plt.plot(mean_amp, '.')
plt.xlabel('Baseline Index')
plt.ylabel('Mean Amplitude')
plt.title('Mean Amplitude per Baseline')
plt.show()

# Check for zeros
zeros = np.sum(mean_amp == 0)
print(f"Number of dead baselines (zero amp): {zeros} / {len(mean_amp)}")

In [ ]:
# Load Bandpass Table
import glob
bp_tables = glob.glob(f"{ms_path}*b") + glob.glob(f"{ms_path}*B")
print(f"Found Bandpass Tables: {bp_tables}")

if bp_tables:
    bp_table = bp_tables[0]
    t = table(bp_table, ack=False)
    flags = t.getcol('FLAG')
    gains = t.getcol('CPARAM')
    t.close()
    
    # Gains shape: (npol, nchan, nant) usually? Or (npol, nchan, nant, nspw)?
    # CASA tables vary.
    print(f"Gains Shape: {gains.shape}")
    print(f"Flags Shape: {flags.shape}")
    
    # Calculate flag percentage
    flag_pct = np.sum(flags) / flags.size * 100
    print(f"Total Flagged Solutions: {flag_pct:.2f}%")
    
    # Plot Amplitudes for one antenna
    amp = np.abs(gains)
    plt.figure(figsize=(10, 4))
    plt.imshow(amp[0, :, :], aspect='auto', interpolation='none', vmin=0, vmax=2)
    plt.colorbar(label='Gain Amp')
    plt.xlabel('Antenna/Row')
    plt.ylabel('Channel')
    plt.title('Bandpass Amplitudes (Pol 0)')
    plt.show()
else:
    print("No bandpass table found to inspect.")